In [1]:
import os
import numpy as np
import tensorflow as tf

from ml.config.train import cfg as cfg_train
from ml.config.test import cfg as cfg_test
from ml.dataset import odgt2train, odgt2test
from ml.models import TASK_MODEL_DICT, ModelBuilder, OptimizerBuilder, LRScheduleBuilder, TimingCallback

from sklearn.metrics import classification_report

print("All packages imported!")

2024-01-10 14:12:51.374022: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


All packages imported!


## Training

Example with static classifier.

### Config

In [2]:
# fill in your directory set up here
config_fp = '/home/jonathan/git/pdiot-ml/config/train/static.yaml'
cfg_train.merge_from_file(config_fp)
cfg_train.DATASET.path = '/home/jonathan/git/pdiot-ml/data/sets/pdiot-data/'
cfg_train.TRAIN.path = '/home/jonathan/git/pdiot-ml/ckpt/static/'   
print(cfg_train)

DATASET:
  LIST:
    train: train_static_pdiot-data.odgt
    val: val_static_pdiot-data.odgt
  num_classes: 5
  path: /home/jonathan/git/pdiot-ml/data/sets/pdiot-data/
MODEL:
  ARCH:
    LSTM:
      hidden_size: 0
      num_layers: 0
    MLP:
      dropout: 1.0
      hidden_size: 0
      num_layers: 0
  INPUT:
    format: summary
    sensor: all
    window_size: 50
TRAIN:
  DATA:
    batch_size: 128
    overlap_size: 0
  FN:
    config: config.yaml
    history: history.csv
    log: log.txt
    weight: weights.hdf5
  LEN:
    early_stop: 5
    num_epoch: 5
  LR:
    gamma: 0.1
    schedule: step
    step_size: 40
  OPTIM:
    lr: 0.01
    momentum: 0.9
    optim: adam
    weight_decay: 0.0005
  path: /home/jonathan/git/pdiot-ml/ckpt/static/


### Data

In [3]:
# experiment 
train_odgt_fp = os.path.join(cfg_train.DATASET.path, cfg_train.DATASET.LIST.train)
val_odgt_fp = os.path.join(cfg_train.DATASET.path, cfg_train.DATASET.LIST.val)

train_X, train_y = odgt2train(train_odgt_fp, cfg_train.MODEL.INPUT.window_size, 
                                            cfg_train.TRAIN.DATA.overlap_size)
val_X, val_y = odgt2train(val_odgt_fp, cfg_train.MODEL.INPUT.window_size, 
                                      cfg_train.TRAIN.DATA.overlap_size)

### Optimization

In [4]:
model = ModelBuilder.build_classifier(cfg_train.MODEL, '', cfg_train.DATASET.num_classes)
optimizer = OptimizerBuilder.build_optimizer(cfg_train.TRAIN.OPTIM)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
metrics = [
    tf.keras.metrics.SparseCategoricalAccuracy(name='acc'),
]

lr_scheduler = LRScheduleBuilder.build_scheduler(cfg_train.TRAIN.LR)
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=cfg_train.TRAIN.LEN.early_stop)

#checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#    filepath=os.path.join(cfg_train.TRAIN.path, 'weights.hdf5'),
#    save_weights_only=True,
#    monitor='val_acc',
#    mode='max',
#    save_best_only=True)

#timing_callback = TimingCallback()

#history_callback = tf.keras.callbacks.CSVLogger(
#    os.path.join(cfg_train.TRAIN.path, 'history.csv'), 
#    separator=',', 
#    append=False)

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
history = model.fit(train_X, train_y, validation_data=(val_X, val_y), 
          epochs=cfg_train.TRAIN.LEN.num_epoch, 
          batch_size=cfg_train.TRAIN.DATA.batch_size,
          callbacks=[lr_callback, 
                     early_stop_callback])
#                     checkpoint_callback,
#                     timing_callback,
#                     history_callback])

Epoch 1/5


2024-01-10 14:12:58.303200: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


297/297 [==============================] - 2s 5ms/step - loss: 0.8095 - acc: 0.8207 - val_loss: 0.3012 - val_acc: 0.9850 - lr: 0.0100
Epoch 2/5
297/297 [==============================] - 1s 5ms/step - loss: 0.2106 - acc: 0.9815 - val_loss: 0.1707 - val_acc: 0.9855 - lr: 0.0100
Epoch 3/5
297/297 [==============================] - 2s 6ms/step - loss: 0.1395 - acc: 0.9841 - val_loss: 0.1342 - val_acc: 0.9910 - lr: 0.0100
Epoch 4/5
297/297 [==============================] - 2s 7ms/step - loss: 0.1154 - acc: 0.9845 - val_loss: 0.1205 - val_acc: 0.9925 - lr: 0.0100
Epoch 5/5
297/297 [==============================] - 2s 6ms/step - loss: 0.1046 - acc: 0.9849 - val_loss: 0.1131 - val_acc: 0.9925 - lr: 0.0100


## Testing

Example for task 1 with subject 29. 

### Config

In [5]:
# fill in your directory set up here
project_dp = '/home/jonathan/git/pdiot-ml/'
config_fp = os.path.join(project_dp, 'config/test/task_1.yaml')
cfg_test.merge_from_file(config_fp)
cfg_test.TEST.subject = 30
cfg_test.DATASET.path = '/home/jonathan/git/pdiot-ml/data/sets/pdiot-data/'
cfg_test.TEST.path = '/home/jonathan/git/pdiot-ml/ckpt/test/task_1'   
print(cfg_test)

DATASET:
  path: /home/jonathan/git/pdiot-ml/data/sets/pdiot-data/
  task: 1
MODEL:
  CONFIG:
    breath: 
    dynamic: config/train/task_1/dynamic.yaml
    motion: config/train/task_1/motion.yaml
    static: config/train/task_1/static.yaml
  INPUT:
    window_size: 50
TEST:
  DATA:
    batch_size: 128
    overlap_size: 0
  path: /home/jonathan/git/pdiot-ml/ckpt/test/task_1
  subject: 30


### Data

In [6]:
# experiment 
test_odgt_fp = os.path.join(cfg_test.DATASET.path, f'full_t{cfg_test.DATASET.task}_pdiot-data.odgt')
test_dict = odgt2test(test_odgt_fp, cfg_test.DATASET.task,
                                    cfg_test.TEST.subject, 
                                    cfg_test.MODEL.INPUT.window_size, 
                                    cfg_test.TEST.DATA.overlap_size)

### Optimization

In [7]:
components = TASK_MODEL_DICT[cfg_test.DATASET.task]
model_dict = {}
for component in components:
    component_cfg_fp = os.path.join(project_dp, cfg_test.MODEL.CONFIG[component])
    cfg_train.merge_from_file(component_cfg_fp)

    model = ModelBuilder.build_classifier(cfg_train.MODEL, '', cfg_train.DATASET.num_classes)
    optimizer = OptimizerBuilder.build_optimizer(cfg_train.TRAIN.OPTIM)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(ignore_class=-1)
    metrics = [
        tf.keras.metrics.SparseCategoricalAccuracy(name='acc'),
    ]

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    history = model.fit(test_dict['train']['X'], test_dict['train'][component], 
                        epochs=cfg_train.TRAIN.LEN.num_epoch, 
                        batch_size=cfg_test.TEST.DATA.batch_size)
    
    model_dict[component] = model

Epoch 1/5
97/97 [==============================] - 1s 5ms/step - loss: 0.2789 - acc: 0.9224
Epoch 2/5
97/97 [==============================] - 1s 5ms/step - loss: 0.1350 - acc: 0.9766
Epoch 3/5
97/97 [==============================] - 1s 5ms/step - loss: 0.1017 - acc: 0.9795
Epoch 4/5
97/97 [==============================] - 0s 4ms/step - loss: 0.0868 - acc: 0.9823
Epoch 5/5
97/97 [==============================] - 0s 3ms/step - loss: 0.0798 - acc: 0.9832
Epoch 1/20


2024-01-10 14:13:14.244103: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-01-10 14:13:14.245304: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-01-10 14:13:14.246286: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

97/97 [==============================] - 4s 19ms/step - loss: 1.6175 - acc: 0.1649
Epoch 2/20
97/97 [==============================] - 2s 20ms/step - loss: 1.4099 - acc: 0.2070
Epoch 3/20
97/97 [==============================] - 2s 22ms/step - loss: 1.3314 - acc: 0.2242
Epoch 4/20
97/97 [==============================] - 2s 24ms/step - loss: 1.2821 - acc: 0.2379
Epoch 5/20
97/97 [==============================] - 2s 24ms/step - loss: 1.2494 - acc: 0.2441
Epoch 6/20
97/97 [==============================] - 2s 18ms/step - loss: 1.2247 - acc: 0.2509
Epoch 7/20
97/97 [==============================] - 2s 23ms/step - loss: 1.2019 - acc: 0.2561
Epoch 8/20
97/97 [==============================] - 2s 21ms/step - loss: 1.1769 - acc: 0.2601
Epoch 9/20
97/97 [==============================] - 2s 21ms/step - loss: 1.1554 - acc: 0.2656
Epoch 10/20
97/97 [==============================] - 2s 26ms/step - loss: 1.1389 - acc: 0.2719
Epoch 11/20
97/97 [==============================] - 2s 19ms/step - lo

### Evaluation

In [8]:
model = ModelBuilder.build_hierarchical_classifier(cfg_test.DATASET.task, model_dict)

# make predictions on test data using the model(s)
pred = model(test_dict['val']['X'])

# make classification report
report = classification_report(test_dict['val']['y'], pred)

# show results
print(f'Task {cfg_test.DATASET.task} - Subject {cfg_test.TEST.subject}') 
print(report)

Task 1 - Subject 30
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        30
           1       1.00      1.00      1.00        15
           2       1.00      1.00      1.00        15
           3       1.00      1.00      1.00        15
           4       1.00      1.00      1.00        15
           5       0.38      0.40      0.39        15
           6       0.70      0.93      0.80        15
           7       0.71      0.36      0.48        14
           8       0.38      0.40      0.39        15
           9       0.38      0.33      0.36        15
          10       0.71      0.80      0.75        15

    accuracy                           0.77       179
   macro avg       0.75      0.75      0.74       179
weighted avg       0.77      0.77      0.76       179



2024-01-10 14:14:03.663628: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-01-10 14:14:03.665125: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-01-10 14:14:03.666332: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus